In [99]:
import pandas as pd
import openai

# Load CSV file
file_path = "../PPR-ALL.csv"
df = pd.read_csv(file_path)

# Define rule-based insights extraction
def rule_based_insights(description, county):
    urban_counties = ["Dublin", "Cork", "Galway", "Limerick", "Waterford"]
    suburban_counties = ["Meath", "Kildare", "Wicklow", "Louth"]
    rural_counties = ["Mayo", "Donegal", "Kerry", "Clare", "Roscommon"]

    if county in urban_counties:
        neighborhood_type = "Urban"
        demand_level = "High"
    elif county in suburban_counties:
        neighborhood_type = "Suburban"
        demand_level = "Medium"
    else:
        neighborhood_type = "Rural"
        demand_level = "Low"

    suitability = "Families" if "New" in description else "Investors"
    proximity_to_schools = "Yes" if neighborhood_type != "Rural" else "No"
    proximity_to_public_transport = "Yes" if neighborhood_type == "Urban" else "No"

    return neighborhood_type, demand_level, suitability, proximity_to_schools, proximity_to_public_transport

# Apply function to dataset
df[['neighborhood_type', 'demand_level', 'suitability', 'proximity_to_schools', 'proximity_to_public_transport']] = df.apply(
    lambda row: rule_based_insights(row['Description of Property'], row['County']), axis=1, result_type="expand"
)

# Save the updated dataset
updated_file_path = "PPR-ALL-updated.csv"
df.to_csv(updated_file_path, index=False)

print(f"Updated file saved as {updated_file_path}")


/var/folders/36/c1xr8mpj3ysczntmtbsctlz80000gn/T/ipykernel_41621/442094442.py:6: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Updated file saved as PPR-ALL-updated.csv


# Adding the Neighbourhood type based on the address - TODO LATER

In [88]:
import requests
import ssl
import certifi
from geopy.geocoders import Nominatim

def get_location(address):
    """Convert address to latitude & longitude."""

    # Force Python to use the certifi SSL certificates
    ssl_context = ssl.create_default_context(cafile=certifi.where())

    # Pass the SSL context to Nominatim
    geolocator = Nominatim(user_agent="geoapi", timeout=10, scheme="https", ssl_context=ssl_context)
    # geolocator = Nominatim(user_agent="geoapi")
    location = geolocator.geocode(address)
    return (location.latitude, location.longitude) if location else (None, None)

def get_historical_neighborhood(lat, lon, year):
    # Example: Use an API to check historical land use classification
    response = requests.get(f"http://api.example.com/historical_landuse?lat={lat}&lon={lon}&year={year}")
    data = response.json()
    
    return data.get("land_type", "Unknown")  # Returns "Urban", "Suburban", or "Rural"

# Example usage
address = "5 Braemor Drive, Churchtown, Dublin, Ireland"
year = 2010
latitude, longitude = get_location(address)
# neighborhood_type = get_historical_neighborhood(latitude, longitude, year)

# print(f"In {year}, the neighborhood type for {address} was: {neighborhood_type}")


In [103]:
import pandas as pd

def load_and_clean_data(file_path):
    df = pd.read_csv(file_path, encoding="ISO-8859-1")
    
    # Convert 'Date of Sale' to datetime
    df["Date of Sale"] = pd.to_datetime(df["Date of Sale (dd/mm/yyyy)"], format="%d/%m/%Y", errors="coerce")
    
    # Convert 'Price (€)' to numeric
    df["Price (€)"] = df["Price"].replace('[^\d.]', '', regex=True).astype(float)
    
    # Drop missing values
    df = df.dropna(subset=["Price (€)", "County"])

    # Extract year and month
    df["Year"] = df["Date of Sale"].dt.year
    df["Month"] = df["Date of Sale"].dt.month

    return df

# Load the updated Data

In [144]:
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor  
from sklearn.model_selection import cross_val_score, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor


df = load_and_clean_data("../PPR-ALL.csv")
print(f"Size:{df.size}, Types: {df.dtypes}, Null: {df.isnull().sum()} describe:{df.describe()}")
df.head()

/var/folders/36/c1xr8mpj3ysczntmtbsctlz80000gn/T/ipykernel_41621/96373143.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, encoding="ISO-8859-1")


Size:9260901, Types: Date of Sale (dd/mm/yyyy)            object
Address                              object
County                               object
Eircode                              object
Price                                object
Not Full Market Price                object
VAT Exclusive                        object
Description of Property              object
Property Size Description            object
Date of Sale                 datetime64[ns]
Price (€)                           float64
Year                                  int32
Month                                 int32
dtype: object, Null: Date of Sale (dd/mm/yyyy)         0
Address                           0
County                            0
Eircode                      532888
Price                             0
Not Full Market Price             0
VAT Exclusive                     0
Description of Property           0
Property Size Description    659546
Date of Sale                      0
Price (€)                 

,Date of Sale (dd/mm/yyyy),Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Date of Sale,Price (€),Year,Month
0,01/01/2010,"5 Braemor Drive, Churchtown, Co.Dublin",Dublin,NaN,"ï¿½343,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,2010-01-01,343000.0,2010,1
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",Laois,NaN,"ï¿½185,000.00",No,Yes,New Dwelling house /Apartment,greater than or equal to 38 sq metres and less...,2010-01-03,185000.0,2010,1
2,04/01/2010,"1 Meadow Avenue, Dundrum, Dublin 14",Dublin,NaN,"ï¿½438,500.00",No,No,Second-Hand Dwelling house /Apartment,NaN,2010-01-04,438500.0,2010,1
3,04/01/2010,"1 The Haven, Mornington",Meath,NaN,"ï¿½400,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,2010-01-04,400000.0,2010,1
4,04/01/2010,"11 Melville Heights, Kilkenny",Kilkenny,NaN,"ï¿½160,000.00",No,No,Second-Hand Dwelling house /Apartment,NaN,2010-01-04,160000.0,2010,1


# Remove all Rows with Null Entries

In [145]:
df.dropna(subset=["Property Size Description"], inplace=True)

In [147]:

# Feature engineering
df["Year"] = df["Date of Sale"].dt.year
df["Month"] = df["Date of Sale"].dt.month

# Preprocess dataset (Feature Engineering steps)
label_enc = LabelEncoder()
df["County"] = label_enc.fit_transform(df["County"])
df["Not Full Market Price"] = label_enc.fit_transform(df["Not Full Market Price"])
df["VAT Exclusive"] = label_enc.fit_transform(df["VAT Exclusive"])
df["Description of Property"] = label_enc.fit_transform(df["Description of Property"])
# df["neighborhood_type"] = label_enc.fit_transform(df["neighborhood_type"])
# df["suitability"] = label_enc.fit_transform(df["suitability"])
# df["proximity_to_schools"] = label_enc.fit_transform(df["proximity_to_schools"])
# df["proximity_to_public_transport"] = label_enc.fit_transform(df["proximity_to_public_transport"])
# df["demand_level"] = label_enc.fit_transform(df["demand_level"])


X = df.drop(columns=["Price", "Price (€)", "Date of Sale", "Date of Sale (dd/mm/yyyy)",  "Address", "Eircode", "Property Size Description"])  #drop non-numeric columns
y = df["Price (€)"]

df.head()



,Date of Sale (dd/mm/yyyy),Address,County,Eircode,Price,Not Full Market Price,VAT Exclusive,Description of Property,Property Size Description,Date of Sale,Price (€),Year,Month
1,03/01/2010,"134 Ashewood Walk, Summerhill Lane, Portlaoise",10,NaN,"ï¿½185,000.00",0,1,0,greater than or equal to 38 sq metres and less...,2010-01-03,185000.0,2010,1
10,04/01/2010,"16 Aisling Geal, Fr. Russell Road",12,NaN,"ï¿½110,000.00",0,0,0,greater than or equal to 38 sq metres and less...,2010-01-04,110000.0,2010,1
14,04/01/2010,"25 Lavallin Drive, Whitechurch, Cork",3,NaN,"ï¿½352,000.00",0,1,0,greater than 125 sq metres,2010-01-04,352000.0,2010,1
20,04/01/2010,"48 KILLIANS COURT, MULLAGH",1,NaN,"ï¿½122,000.00",0,1,0,greater than 125 sq metres,2010-01-04,122000.0,2010,1
21,04/01/2010,"49 ballynakelly green, newcastle",5,NaN,"ï¿½499,600.00",0,1,0,greater than 125 sq metres,2010-01-04,499600.0,2010,1


In [148]:
print(f"X by Y size: {X.shape}, {y.shape}")

X by Y size: (52831, 6), (52831,)


In [149]:
X.head()

,County,Not Full Market Price,VAT Exclusive,Description of Property,Year,Month
1,10,0,1,0,2010,1
10,12,0,0,0,2010,1
14,3,0,1,0,2010,1
20,1,0,1,0,2010,1
21,5,0,1,0,2010,1


# Model Cross-Validation

In [150]:
# Define models
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "Lasso Regression": Lasso(alpha=0.01),
    "Decision Tree": DecisionTreeRegressor(max_depth=10),
    "Random Forest": RandomForestRegressor(n_estimators=100),
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100),
    "XGBoost": XGBRegressor(n_estimators=100, learning_rate=0.1),
    "LightGBM": LGBMRegressor(n_estimators=100),
    # "CatBoost": CatBoostRegressor(iterations=100, verbose=0)
}

# Perform cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)
results = {}

for name, model in models.items():
    scores = cross_val_score(model, X, y, scoring="neg_root_mean_squared_error", cv=kf)
    results[name] = np.mean(-scores)  # Convert negative RMSE to positive


# Display results
for model, rmse in sorted(results.items(), key=lambda x: x[1]):
    print(f"{model}: RMSE = {rmse:.2f}")


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001943 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 42264, number of used features: 5
[LightGBM] [Info] Start training from score 254349.416534
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000902 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 57
[LightGBM] [Info] Number of data points in the train set: 42265, number of used features: 5
[LightGBM] [Info] Start training from score 249228.236707
[LightGBM] [Warning] Fou

# Hyperparameter Tuning?
Each model has hyperparameters (like tree depth, number of estimators, learning rate, etc.) that greatly affect accuracy. We'll optimize them using Grid Search or Random Search.
Grid Search tries every combination of parameters, which can be slow but guarantees the best values.
Instead of testing all combinations, RandomizedSearchCV picks random values, making it much faster

In [40]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Define the parameter grid
param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [3, 5, 7],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "subsample": [0.6, 0.8, 1.0]
}

# Initialize the model
gbr = GradientBoostingRegressor()

# Run Grid Search
grid_search = GridSearchCV(gbr, param_grid, scoring="neg_root_mean_squared_error", 
                           cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X, y)

# Print best parameters and RMSE
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best RMSE: {-grid_search.best_score_:.2f}")

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.6; total time=  43.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.6; total time=  44.2s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.6; total time=  44.7s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.6; total time=  44.7s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.6; total time=  44.8s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.8; total time=  50.0s
[CV] END learning_rate=0.01, max_depth=3, min_samples_leaf=1, min_samples_split=2, n_estimators=100, subsample=0.8; total time

In [166]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Define X and y (Make sure you have preprocessed your dataset)
# X = your_features
# y = your_target_variable

# Define the model
xgb = XGBRegressor(objective="reg:squarederror", random_state=42)

# Define the parameter grid
param_grid = {
    "n_estimators": np.arange(20, 100, 20),
    "learning_rate": [0.01, 0.05, 0.1, 0.2, 0.3],
    "max_depth": np.arange(3, 15, 2),
    "min_child_weight": [1, 3, 5, 7],
    "subsample": [0.6, 0.7, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.6, 0.7, 0.8, 0.9, 1.0],
}

# Run RandomizedSearchCV for fast tuning
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=20,  # Lower for faster tuning, increase for better results
    scoring="neg_mean_absolute_error",
    cv=5,
    verbose=1,
    n_jobs=-1
)

# Fit the model
random_search.fit(X, y)

# Best parameters
best_params = random_search.best_params_
print("Best Parameters:", best_params)

# Train the final model with best parameters
best_xgb = XGBRegressor(**best_params, objective="reg:squarederror", random_state=42)
best_xgb.fit(X, y)

# Predictions
y_pred = best_xgb.predict(X)

# Evaluate performance
mae = mean_absolute_error(y, y_pred)
mse = mean_squared_error(y, y_pred)
rmse = np.sqrt(mse) 
r2 = r2_score(y, y_pred)

print(f"MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")

Fitting 5 folds for each of 20 candidates, totalling 100 fits
Best Parameters: {'subsample': 0.7, 'n_estimators': np.int64(40), 'min_child_weight': 1, 'max_depth': np.int64(7), 'learning_rate': 0.05, 'colsample_bytree': 0.7}
MAE: 104744.6495, MSE: 378301362522.1157, RMSE: 615062.0802, R²: 0.0942


# Train Model - XG Regressor

In [151]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.head()


,County,Not Full Market Price,VAT Exclusive,Description of Property,Year,Month
332152,5,0,1,0,2018,9
316983,16,0,1,0,2018,6
56636,5,0,1,0,2012,10
270162,8,0,1,0,2017,8
312521,3,0,1,0,2018,5


In [156]:
# Initialize and train the model
model = XGBRegressor(n_estimators=100, learning_rate=0.1)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.1, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

# Measure Model Performance

In [157]:
# Make predictions
y_pred = model.predict(X_test)

# Compute regression metrics
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  # Root Mean Squared Error
r2 = r2_score(y_test, y_pred)


print("🔍 Regression Model Performance:")
print(f"✅ Mean Absolute Error (MAE): {mae:.2f}")
print("MAE Measures the average absolute difference between actual and predicted prices. Lower is better. A MAE of €24,500 means, on average, the model's predictions are €24,500 off.")
print(f"✅ Mean Squared Error (MSE): {mse:.2f}")
print("Similar to MAE, but squares errors, making large errors more significant. Lower is better. If MSE = 1.24e+10, it means that large price deviations are penalized.")
print(f"✅ Root Mean Squared Error (RMSE): {rmse:.2f}")
print("RMSE is the square root of MSE, providing error in the same units as the target variable. Lower is better. If RMSE = €35,000, it indicates that the model's predictions are off by about €35,000 on average.")
print(f"✅ R² Score: {r2:.2f}")
print("Range: 0 (bad) → 1 (perfect fit). \n If R² = 0.87, this means that 87% of the variation in house prices is explained by the model. \n Higher is better. A low R² means the model is not capturing useful patterns.")


🔍 Regression Model Performance:
✅ Mean Absolute Error (MAE): 98965.15
MAE Measures the average absolute difference between actual and predicted prices. Lower is better. A MAE of €24,500 means, on average, the model's predictions are €24,500 off.
✅ Mean Squared Error (MSE): 62228286642.42
Similar to MAE, but squares errors, making large errors more significant. Lower is better. If MSE = 1.24e+10, it means that large price deviations are penalized.
✅ Root Mean Squared Error (RMSE): 249455.98
RMSE is the square root of MSE, providing error in the same units as the target variable. Lower is better. If RMSE = €35,000, it indicates that the model's predictions are off by about €35,000 on average.
✅ R² Score: 0.09
Range: 0 (bad) → 1 (perfect fit). 
 If R² = 0.87, this means that 87% of the variation in house prices is explained by the model. 
 Higher is better. A low R² means the model is not capturing useful patterns.


This is still pretty terrible so I am trying the hybrid approach

In [143]:
import os
from openai import OpenAI

# Set the API key

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key=os.getenv("OPENAI_API_KEY"))


def extract_insights(property_description):
    prompt = f"""
    Analyze the following property listing and extract insights:
    
    "{property_description}"
    
    Provide the following structured output in JSON format:
    - "neighborhood_type": (Urban, Suburban, Rural)
    - "demand_level": (High, Medium, Low)
    - "suitability": (Families, Investors, Retirees)
    - "proximity_to_schools": (Yes/No)
    - "proximity_to_public_transport": (Yes/No)
    """

    response = client.completions.create(
        model="gpt-3.5-turbo",  # Use a free LLM like Mistral-7B if needed
        prompt=prompt,
    )

    return response # Extract the text from the response


# Example usage
property_description = "3-bedroom house in Dublin, located in a quiet neighborhood with schools nearby. Great for families. Close to public transport."
insights = extract_insights(property_description)
print(insights.choices[0].text)  # Print the structured output

{
    "neighborhood_type": "Suburban",
    "demand_level": "Medium",
    "suitability": "Families",
    "proximity_to_schools": "Yes",
    "proximity_to_public_transport": "Yes"
}
